In [ ]:
import torch
import numpy as np
import random
import time
from tqdm import tqdm  # 
from pathlib import Path


seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
torch.backends.cudnn.benchmark = True


if torch.cuda.is_available():
    print(f"cuda:True")
    for i in range(torch.cuda.device_count()):
        device = torch.device(f"cuda:{i}")
        properties = torch.cuda.get_device_properties(device)
        print(f"GPU {i}:{properties.name}:{properties.total_memory/1024/1024/1024:.2f}GB")
else:
    print(f"cuda:False")
    device = torch.device("cpu")


from models_LN import FNN_LN, dataset, one_hot, cross_entropy_loss, linear, linear_d, tanh, tanh_d, sign, sign_d, sigmoid, sigmoid_d, relu, relu_d, relu6, relu6_d, softmax, softmax_d

torch.set_grad_enabled(False)

cuda:True
GPU 0:NVIDIA RTX A5000:23.99GB


In [2]:

class Config:
    def __init__(self):
        self.n_input = 28 * 28  # 
        self.n_hidd = 16     # 
        self.n_out = 10      # 

        self.nL_hidd = 2

        self.sc_forward = 1
        self.sc_bias = 0
        self.sc_back = 1

        self.f = tanh #  relu tanh
        self.fd = 0 #tanh_d # relu_d tanh_d

        self.train_eta_global = 1e-3  #
        self.train_n_epochs = 100     # 
        self.train_batch_size = 20     # 
        self.train_task = "MNIST" #  "MNIST" "FMNIST" "CIFAR10"
        self.train_tmethod = ''     # 'FA','PC','LRA','BP'

        self.biasLearning = False
        
        self.EP_It2sta = 8
        self.EP_nudge = 1
        
config = Config()

In [3]:
train_loader, valid_loader, test_loader, config = dataset(config)
config.test_batch_size = 500
train_loader_gpu = [(data.to(device), target.to(device)) for data, target in train_loader]
valid_loader_gpu = [(data.to(device), target.to(device)) for data, target in valid_loader]
test_loader_gpu = [(data.to(device), target.to(device)) for data, target in test_loader]

In [4]:
##
ParaName = f'First_symm_LN_t2sta2_bs20_lrsame'
Einfo = ''

res_path = Path(f"./Res/{ParaName}-{config.train_task}")
M_range = [0.01] # 0.001, 0.01, 0.1, 0.25, 1, 2, 4 fbsc  # 0.01, 0.1, 0.25, 0.5, 1, 2 SRsc
ParaRange = [3] # 2,3,5,8
multitest = 5 
config.save = True
config.ff = softmax
config.ffd = 0 #softmax_d
# -=--------------------------
config.train_tmethod = 'EP'
# config.nL_hidd = 2
config.n_hidd = 512     # 

config.train_n_epochs = 100  # 

config.EP_symm_W = True
config.adam = False
config.EP_b_learn = False
config.EP_f_sc = 1
config.EP_nudge = 0.1
# config.EP_b_sc = 0.25

In [5]:
# -=--------------------------
res_path.mkdir(parents=True, exist_ok=True)
res = np.zeros((len(M_range),len(ParaRange),(multitest),6,config.train_n_epochs))

zall_list = [[[] for _ in range(len(ParaRange))] for _ in range(len(M_range))]
eall_list = [[[[] for _ in range((multitest))] for _ in range(len(ParaRange))] for _ in range(len(M_range))]

In [6]:
current_time = time.strftime("%Y%m%d-%H%M%S", time.localtime())
print(f'Current Time: {current_time}')
with open(res_path/"res.txt", "a") as f: 
    f.write(f"===================================================================")       
    f.write(f"Start Time: {current_time}\r\n")

Current Time: 20250721-172600


In [7]:
for iPara in range( len(ParaRange) ):
    for iM in range( len(M_range) ):
        for  imultitest in range((multitest)):
            config.EP_b_sc = M_range[iM]
            config.nL_hidd = ParaRange[iPara]

            #-----------------
            config.EP_It2sta = 10 if config.nL_hidd==2 else 18
            config.EP_It2sta2 = 5 if config.nL_hidd==2 else 10
            # ---------------------------------
            model = FNN_LN(config).to(device)

            taskinfo = f'EP_L{config.nL_hidd}_{config.train_task}_{ParaName}-EP_b_sc{M_range[iM]}-EP_It2sta{config.EP_It2sta}-{Einfo}'
            print(taskinfo)

            ## Train

            for epoch in range(config.train_n_epochs):
                losssum = 0
                train_correct = 0
                train_total = 0
                random.shuffle(train_loader_gpu)
                with tqdm(total=len(train_loader_gpu), desc=f'Epoch {epoch + 1}/{config.train_n_epochs}: ', unit='batch', ncols=90, mininterval=1, colour='blue') as pbar:
                    for i, (images, labels) in enumerate(train_loader_gpu):
                        # 
                        images = images.view(-1, config.n_input)
                        labels_one_hot = one_hot(labels, config.n_out, device=device)
                        # 
                        outputs = model.forward(images)
                        _, predicted = torch.max(outputs, 1)
                        # 
                        loss = cross_entropy_loss(outputs, labels_one_hot.to(device))
                        losssum += loss.item()
                        #
                        outputs = model.backward(images, labels_one_hot, outputs)
                        # 
                        model.update_weights_adam() if config.adam else model.update_weights(lr=[0.1,0.1,0.1,0.1]) #model.update_weights_adam() model.update_weights()
                        # 
                        if i % 100 == 0: 
                            pbar.set_postfix({'loss': f'{losssum/(i+1):.6f}'})  
                            pbar.update(100)

                # SR, ftmle, mle, zall = model.Lyapunov_EP(images[-1:,:], y=None, t_e=500, ret_zall=True)
                # if imultitest==0: zall_list[iM][iPara].append(zall)

                # terr = model.ret_error()
                # eall_list[iM][iPara][imultitest].append(terr)

                # # train set ------------------------------------------------------------
                # losssum = 0
                # for images, labels in valid_loader_gpu:
                #     images = images.view(-1, config.n_input)
                #     labels_one_hot = one_hot(labels, config.n_out, device=device)
                #     # 
                #     outputs = model.forward(images)

                    # loss = cross_entropy_loss(outputs, labels_one_hot.to(device))
                        # losssum += loss.item()
                        train_total += labels.size(0)
                        train_correct += (predicted == labels).sum().item()

                train_accuracy = train_correct / train_total

                # test set -------------------------------------------------------------
                test_correct = 0
                test_total = 0
                # ---------
                for images, labels in test_loader_gpu:
                    images = images.view(-1, config.n_input)
                    # 
                    outputs = model.forward(images)

                    _, predicted = torch.max(outputs, 1)
                    test_total += labels.size(0)
                    test_correct += (predicted == labels).sum().item()

                test_accuracy = test_correct / test_total

                res[iM][iPara][imultitest][0][epoch] = losssum
                res[iM][iPara][imultitest][1][epoch] = train_accuracy
                res[iM][iPara][imultitest][2][epoch] = test_accuracy
                # res[iM][iPara][imultitest][3][epoch] = ftmle
                # res[iM][iPara][imultitest][4][epoch] = mle
                # res[iM][iPara][imultitest][5][epoch] = SR

                current_time = time.strftime("%Y%m%d-%H%M%S", time.localtime())
                print(f'\t Loss: {res[iM][iPara][imultitest][0][epoch] :.3f},\t'
                    f'Train : {res[iM][iPara][imultitest][1][epoch] * 100:.2f}%({np.max(res[iM][iPara][imultitest][1]) * 100:.2f}%),\t'
                    f'Test : {res[iM][iPara][imultitest][2][epoch] * 100:.2f}%({np.max(res[iM][iPara][imultitest][2]) * 100:.2f}%),\t'
                    f'Current Time: {current_time}')
                
                # if 1-train_accuracy <1e-4:
                #     break
            
            # best result
            print(f"{taskinfo} Time: {current_time}, Epochs: {config.train_n_epochs}, Learning Rate: {config.train_eta_global}, "
                f"Best: {max(res[iM][iPara][imultitest][1]) * 100:.2f}%  {max(res[iM][iPara][imultitest][2]) * 100:.2f}%\n")
            
        for attempt in range(5):
            try:
                with open(res_path/"res.txt", "a") as f:
                    trainacct = np.max(res[iM][iPara][:,1,:], axis=-1)
                    testacct = np.max(res[iM][iPara][:,2,:], axis=-1)
                    f.write(f"{taskinfo} Time: {current_time}: "
                            f"train: {np.mean(trainacct) * 100:.2f}%+-{np.std(trainacct, ddof=1)*100:.2f}% ({np.min(trainacct) * 100:.2f}% {np.max(trainacct) * 100:.2f}%)"
                            f"\t test: {np.mean(testacct) * 100:.2f}%+-{np.std(testacct, ddof=1)*100:.2f}% ({np.min(testacct) * 100:.2f}% {np.max(testacct) * 100:.2f}%)\n")

                    for attr_name in dir(config):
                        if not attr_name.startswith('__') and not callable(getattr(config, attr_name)):  
                            attr_value = getattr(config, attr_name)
                            f.write(f'{attr_name}: {attr_value}  ')  
                    f.write(f'\n\n')  

                break  # 
            except PermissionError:
                if attempt < 5 - 1:
                    print(f"Permission denied, retrying in {5} seconds...")
                    time.sleep(5)
                else:
                    raise  # 
                        
                

EP_L3_MNIST_First_symm_LN_t2sta2_bs20_lrsame-EP_b_sc0.01-EP_It2sta18-


Epoch 1/100:   0%|                                            | 0/3000 [00:00<?, ?batch/s]d:\GPUS2share\aEP\models_LN.py:249: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(self.z, device=self.device)
Epoch 1/100: 100%|██████████████████| 3000/3000 [00:35<00:00, 84.13batch/s, loss=0.426320]


	 Loss: 1270.498,	Train : 88.00%(88.00%),	Test : 90.70%(90.70%),	Current Time: 20250721-172636


Epoch 2/100: 100%|██████████████████| 3000/3000 [00:34<00:00, 86.72batch/s, loss=0.320436]


	 Loss: 958.254,	Train : 90.88%(90.88%),	Test : 91.45%(91.45%),	Current Time: 20250721-172711


...

	 Loss: 14.375,	Train : 99.98%(99.99%),	Test : 97.66%(97.81%),	Current Time: 20250721-223420


Epoch 99/100: 100%|█████████████████| 3000/3000 [00:34<00:00, 86.09batch/s, loss=0.004708]


	 Loss: 14.214,	Train : 99.99%(99.99%),	Test : 97.75%(97.81%),	Current Time: 20250721-223456


Epoch 100/100: 100%|████████████████| 3000/3000 [00:34<00:00, 87.23batch/s, loss=0.004553]


	 Loss: 13.697,	Train : 99.98%(99.99%),	Test : 97.80%(97.81%),	Current Time: 20250721-223531
EP_L3_MNIST_First_symm_LN_t2sta2_bs20_lrsame-EP_b_sc0.01-EP_It2sta18- Time: 20250721-223531, Epochs: 100, Learning Rate: 0.001, Best: 99.99%  97.81%



In [8]:
# ---------------------------
current_time = time.strftime("%Y%m%d-%H%M%S", time.localtime())
if config.save: 
    save_dict = {
        "M_range": M_range,
        "ParaRange": ParaRange,
        "multitest": multitest,
        "train_task": config.train_task,
        "res": res,
    }
    np.savez_compressed(res_path / f"{taskinfo}_{current_time}_res",  **save_dict)

    # save_dict = {
    #     f"eall_{i}_{j}_{k}": np.array(eall_list[i][j][k])  # 
    #     for i in range(len(M_range))
    #     for j in range(len(ParaRange))
    #     for k in range(multitest)
    # }
    # np.savez_compressed(res_path / f"{taskinfo}_{current_time}_E", **save_dict)

    # save_dict = {
    #     f"zall_{i}_{j}": np.array(zall_list[i][j])  # 
    #     for i in range(len(M_range))
    #     for j in range(len(ParaRange))
    # }
    # np.savez_compressed(res_path / f"{taskinfo}_{current_time}_dyn", **save_dict)




In [9]:

for iM in range( len(M_range) ):
    for iPara in range( len(ParaRange) ):        
        tt = res[iM][iPara]
        tt = np.max(tt[:,2,:], axis= -1)
        testacctt = np.mean(tt, axis=0)

        tt = res[iM][iPara]
        tt = np.max(tt[:,1,:], axis= -1)
        trainacctt = np.mean(tt, axis=0)

        print(f"{taskinfo} Time: {current_time}: "
            f"train: {np.mean(trainacct) * 100:.2f}%+-{np.std(trainacct, ddof=1)*100:.2f}% ({np.min(trainacct) * 100:.2f}% {np.max(trainacct) * 100:.2f}%)"
            f"\t test: {np.mean(testacct) * 100:.2f}%+-{np.std(testacct, ddof=1)*100:.2f}% ({np.min(testacct) * 100:.2f}% {np.max(testacct) * 100:.2f}%)\n")

EP_L3_MNIST_First_symm_LN_t2sta2_bs20_lrsame-EP_b_sc0.01-EP_It2sta18- Time: 20250721-223531: train: 99.98%+-0.01% (99.97% 99.99%)	 test: 97.83%+-0.13% (97.68% 98.04%)

